# Merging data frames

Merging in Pandas is similar to joining tables with SQL.

The basic idea is to combine the attributes of several dataframes based on a common value.  In this case we are adding new columns to an existing dataframe where with the concat method we were adding new rows.

In [1]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import numpy as np

eagle = gpd.read_file("data/BAEA_Nests.shp")
eagle_sources = pd.read_csv("data/BAEA_sources.csv")
eagle_surveys = pd.read_csv("data/BAEA_surveys.csv")
eagle.head()

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry
0,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607)
1,35,40.158003,-104.551141,INACTIVE LOCATION,35,POINT (-104.55114 40.15800)
2,36,39.929381,-104.970620,ACTIVE NEST,36,POINT (-104.97062 39.92938)
3,37,40.556888,-104.974663,ACTIVE NEST,37,POINT (-104.97466 40.55689)
4,46,40.055709,-105.014636,ACTIVE NEST,46,POINT (-105.01464 40.05571)


The eagle_sources data frame contains 4 rows and is an example of a lookup table.  These were more common in the past when storage was more expensive and it was possible to save considerable space in a database by storing coded values and then linking the codes to text values with a lookup table. In this case the code could be stored as a single byte, where as the text string would require at least 30 bytes.  This level of savings may not be that important with the amount of storage available on modern computers but if you have large datasets it can still be significant.

In this example there are many coded values in the eagle GeoDataFrame for every entry in the eagle_sources data frame.  A many to one relationship

In [2]:
eagle_sources

,code,source
0,1,Colorado Fish Wildlife and Parks
1,2,US Fish and Wildlife Service
2,3,ABC Environmental
3,4,123 Environmental


The eagle_surveys table is an example where there are many entries for each entry in the eagle GeoDataFrame. A one to many relationship. In this case each record in the eagle_sources data frame represents the results of an eagle nest surveys.  Surveys were conducted every week during the eagle breeding season so there are 1997 survey records covering 67 eagle nests.

In [3]:
eagle_surveys

,id,nest,surveyor,date,result
0,6.0,52,Dan Smith,2015-07-11,ACTIVE NEST
1,8.0,54,Mike Miller,2015-05-08,ACTIVE NEST
2,9.0,25,Mike Miller,2015-05-13,INACTIVE NEST
3,10.0,65,Mike Miller,2015-08-21,INACTIVE NEST
4,11.0,33,Mike Miller,2015-06-26,ACTIVE NEST
...,...,...,...,...,...
1992,1997.0,64,Mike Miller,2015-06-26,FLEDGED NEST
1993,1998.0,56,George Baker,2015-08-17,ACTIVE NEST
1994,1999.0,16,Dan Smith,2015-08-30,INACTIVE NEST
1995,2000.0,60,Mike Miller,2015-07-22,INACTIVE NEST


Lets add a code field to the eagles data and populate it with a random number from 1 to 4.

In [4]:
eagle['code']=np.random.randint(1,5,size=67)
eagle.sort_values('nest_id')

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry,code
0,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2
9,2,40.235243,-104.973296,ACTIVE NEST,2,POINT (-104.97330 40.23524),3
10,3,40.385732,-104.828742,ACTIVE NEST,3,POINT (-104.82874 40.38573),4
11,4,40.450020,-104.623042,ACTIVE NEST,4,POINT (-104.62304 40.45002),4
12,5,39.983218,-104.895140,ACTIVE NEST,5,POINT (-104.89514 39.98322),3
...,...,...,...,...,...,...,...
62,68,39.838158,-104.292067,ACTIVE NEST,68,POINT (-104.29207 39.83816),4
63,69,40.385771,-104.601066,ACTIVE NEST,69,POINT (-104.60107 40.38577),4
64,73,39.885310,-104.850000,ACTIVE NEST,73,POINT (-104.85000 39.88531),4
65,75,40.005430,-104.726140,INACTIVE LOCATION,75,POINT (-104.72614 40.00543),3


Now lets merge the eagle table with the eagle_source table into a new table called eagle_merge.  The merge command is called on the Pandas object itself.  It takes two data frames as parameters, the first is considered the left data frame and the second is considered the right data frame.  This will be important later on.  

The how parameter specifies the type of join, possibliities include inner, left, right, and outer which will be familiar to SQL users. We'll discuss this later on.  

The on parameter specifies the field name to join on and can be used when the field name is the same in both tables.  

In [5]:
eagle_merge = pd.merge(eagle, eagle_sources, how='inner', on='code').sort_values('nest_id')
eagle_merge

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry,code,source
0,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service
17,2,40.235243,-104.973296,ACTIVE NEST,2,POINT (-104.97330 40.23524),3,ABC Environmental
34,3,40.385732,-104.828742,ACTIVE NEST,3,POINT (-104.82874 40.38573),4,123 Environmental
35,4,40.450020,-104.623042,ACTIVE NEST,4,POINT (-104.62304 40.45002),4,123 Environmental
18,5,39.983218,-104.895140,ACTIVE NEST,5,POINT (-104.89514 39.98322),3,ABC Environmental
...,...,...,...,...,...,...,...,...
46,68,39.838158,-104.292067,ACTIVE NEST,68,POINT (-104.29207 39.83816),4,123 Environmental
47,69,40.385771,-104.601066,ACTIVE NEST,69,POINT (-104.60107 40.38577),4,123 Environmental
48,73,39.885310,-104.850000,ACTIVE NEST,73,POINT (-104.85000 39.88531),4,123 Environmental
30,75,40.005430,-104.726140,INACTIVE LOCATION,75,POINT (-104.72614 40.00543),3,ABC Environmental


The source field can now be used like any other field in the table, for instance as the index for a pivot table, but it doesn't take up any more space on disk.

In [6]:
pd.pivot_table(eagle_merge, index=['source', 'status'], values='nest_id', aggfunc='count')

nest_id
source                           status                    
123 Environmental                ACTIVE NEST             13
                                 INACTIVE LOCATION        4
ABC Environmental                ACTIVE NEST             10
                                 INACTIVE LOCATION        8
Colorado Fish Wildlife and Parks ACTIVE NEST             10
                                 INACTIVE LOCATION        8
US Fish and Wildlife Service     ACTIVE NEST             11
                                 INACTIVE LOCATION        3

Our new dataset can also be merged again with the eagle_survey data frame. This results in a one to many relationship.  **NOTE** In this example the columns we are joining on have different names so we cannot use the *on* parameter, instead we have to specify the column names for each data frame using the *left_on* and *right_on* parameters.

In [7]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='inner', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry,code,source,id,nest,surveyor,date,result
12,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1623.0,1,Mike Miller,2015-03-15,ACTIVE NEST
3,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,706.0,1,Mike Miller,2015-03-26,INACTIVE NEST
18,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1865.0,1,Mike Miller,2015-04-02,INACTIVE NEST
11,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1513.0,1,Mike Miller,2015-04-07,FLEDGED NEST
4,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,843.0,1,Mike Miller,2015-04-19,ACTIVE NEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,891.0,76,Dan Smith,2015-08-06,FLEDGED NEST
1764,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,1834.0,76,Mike Miller,2015-08-06,INACTIVE NEST
1757,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,1078.0,76,Mike Miller,2015-08-17,ACTIVE NEST
1749,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,184.0,76,George Baker,2015-08-20,FLEDGED NEST


Notice that in this case we end up with 1766 rows even though there are 1997 rows in the eagle_surveys table.  This is because we are doing an "inner join" which means that the resulting dataframe only has a record for cases where there is a match between the left_on and right_on columns.  Another possibility is a "left join" which means that there is at least one record for every row in the left table whether or not there is a corresponding record in the right table.  If there is not a corresponding record in the right table then the field values for the right table will be NaN.

In [8]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='left', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry,code,source,id,nest,surveyor,date,result
12,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1623.0,1,Mike Miller,2015-03-15,ACTIVE NEST
3,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,706.0,1,Mike Miller,2015-03-26,INACTIVE NEST
18,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1865.0,1,Mike Miller,2015-04-02,INACTIVE NEST
11,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,1513.0,1,Mike Miller,2015-04-07,FLEDGED NEST
4,1,40.266073,-104.803547,ACTIVE NEST,1,POINT (-104.80355 40.26607),2,US Fish and Wildlife Service,843.0,1,Mike Miller,2015-04-19,ACTIVE NEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,891.0,76,Dan Smith,2015-08-06,FLEDGED NEST
1764,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,1834.0,76,Mike Miller,2015-08-06,INACTIVE NEST
1757,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,1078.0,76,Mike Miller,2015-08-17,ACTIVE NEST
1749,76,40.090090,-104.878150,INACTIVE LOCATION,76,POINT (-104.87815 40.09009),3,ABC Environmental,184.0,76,George Baker,2015-08-20,FLEDGED NEST


In this case we have the same number of records as we did with an inner join which tells us that there are no records in the left table that have no corresponding record in the right table.

With a "right join" there will be a record for every record in the right table whether or not there is a corresponding record in the left table.  

In [9]:
eagle_merge2 = pd.merge(eagle_merge, eagle_surveys, how='right', left_on='nest_id', right_on='nest').sort_values(['nest_id', 'date'])
eagle_merge2

,postgis_fi,lat_y_dd,long_x_dd,status,nest_id,geometry,code,source,id,nest,surveyor,date,result
1618,1.0,40.266073,-104.803547,ACTIVE NEST,1.0,POINT (-104.80355 40.26607),2.0,US Fish and Wildlife Service,1623.0,1,Mike Miller,2015-03-15,ACTIVE NEST
701,1.0,40.266073,-104.803547,ACTIVE NEST,1.0,POINT (-104.80355 40.26607),2.0,US Fish and Wildlife Service,706.0,1,Mike Miller,2015-03-26,INACTIVE NEST
1860,1.0,40.266073,-104.803547,ACTIVE NEST,1.0,POINT (-104.80355 40.26607),2.0,US Fish and Wildlife Service,1865.0,1,Mike Miller,2015-04-02,INACTIVE NEST
1508,1.0,40.266073,-104.803547,ACTIVE NEST,1.0,POINT (-104.80355 40.26607),2.0,US Fish and Wildlife Service,1513.0,1,Mike Miller,2015-04-07,FLEDGED NEST
838,1.0,40.266073,-104.803547,ACTIVE NEST,1.0,POINT (-104.80355 40.26607),2.0,US Fish and Wildlife Service,843.0,1,Mike Miller,2015-04-19,ACTIVE NEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,1049.0,25,Mike Miller,2015-08-29,FLEDGED NEST
1279,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,1284.0,72,George Baker,2015-08-30,FLEDGED NEST
1388,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,1393.0,14,George Baker,2015-08-30,INACTIVE NEST
1484,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,1489.0,74,Mike Miller,2015-08-30,INACTIVE NEST


We can identify the surveys that have no corresponding record in the eagle table by summarizing the resulting dataframe by nest and getting a count of the nest_id field. 

In [10]:
eagle_merge2.groupby(by='nest').count()['nest_id'].sort_values()

nest
74     0
72     0
71     0
70     0
52     0
      ..
8     35
36    36
53    36
50    37
68    39
Name: nest_id, Length: 76, dtype: int64

As with concatenation, Pandas has a rich ecosystem for merging and joining data and there is a lot of additional functionality that canbe achieved with the merge method as well as a good bit of overlap with other methods.  Again I would refer you to the documentation for specifics on each method and in particular the page in the Pandas user guide on [Merge, Join, Concatenate, etc](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) for more information